#  Download GEO shapefiles

- Create a dataset of MA Zipcode by GEO Location

In [1]:
# Steps performed:
# 1. Define the URL for the TIGER/Line 2025 US ZCTA shapefile ZIP.
# 2. Specify local file paths:
#    - zip_path: where the downloaded ZIP file will be temporarily saved.
#    - extract_dir: directory where the shapefile will be extracted.
# 3. Download the ZIP file using requests and save it locally.
# 4. Extract all contents of the ZIP to the specified extraction directory.
# 5. Print confirmation and list the extracted folder.
# 6. Delete the ZIP file after extraction to save disk space, keeping only the shapefile.
#

import os
import shutil
import zipfile

import geopandas as gpd
import requests


def download_and_extract_tiger(
    input_url, output_path, base_url="https://www2.census.gov/geo/tiger/TIGER2025"
):
    """
    Download and extract a TIGER/Line ZIP file.

    Parameters
    ----------
    input_url : str
        Path after TIGER2025 (e.g. "ZCTA520/tl_2025_us_zcta520.zip")

    output_path : str
        Directory where extracted files will be placed

    base_url : str
        Base Census TIGER URL (default: TIGER2025)
    """

    # Build full URL
    full_url = f"{base_url}/{input_url}"

    # Ensure output directory exists
    os.makedirs(output_path, exist_ok=True)

    # Local zip path
    zip_name = os.path.basename(input_url)
    zip_path = os.path.join(output_path, zip_name)

    print(f"Downloading: {full_url}")

    # Download
    r = requests.get(full_url, stream=True)
    r.raise_for_status()

    with open(zip_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Download complete.")

    # Extract
    print("Extracting...")
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(output_path)

    print(f"Extracted to: {output_path}")

    # Remove zip
    os.remove(zip_path)
    print("ZIP file removed.")

In [ ]:
# --------------------------------------------------------------------------------------------------
# Cell: Download Massachusetts ZCTA shapefile
#
# This cell downloads the full US ZIP Code Tabulation Area (ZCTA) shapefile for 2025 from the
# US Census Bureau TIGER/Line repository, extracts it locally, and prepares it for further analysis.
# --------------------------------------------------------------------------------------------------


download_and_extract_tiger(
    input_url="ZCTA520/tl_2025_us_zcta520.zip",
    output_path="../data/external/tl_2025_us_zcta520",
)

In [ ]:
# --------------------------------------------------------------------------------------------------
# Cell: Download US States shapefile (TIGER/Line 2025)
#
# This cell downloads the US States shapefile for 2025 from the US Census Bureau TIGER/Line repository,
# extracts it locally, and prepares it for use in spatial operations such as joining with ZCTA polygons.
# --------------------------------------------------------------------------------------------------

download_and_extract_tiger(
    input_url="/STATE/tl_2025_us_state.zip",
    output_path="../data/external/tl_2025_us_state",
)

In [ ]:
download_and_extract_tiger(
    input_url="/PLACE/tl_2025_25_place.zip",
    output_path="../data/external/tl_2025_ma_place",
)

In [ ]:
# --------------------------------------------------------------------------------------------------
# Cell: Extract Massachusetts ZCTAs via spatial join
#
# This cell performs a spatial join between the full US ZCTA shapefile and the US States shapefile
# to extract only the ZIP Code Tabulation Areas (ZCTAs) that intersect Massachusetts.
# The resulting shapefile is saved in a separate folder for future use.
# --------------------------------------------------------------------------------------------------


# Paths
base_dir = "../data/external"
zcta_shp = os.path.join(base_dir, "tl_2025_us_zcta520", "tl_2025_us_zcta520.shp")
states_shp = os.path.join(base_dir, "tl_2025_us_state", "tl_2025_us_state.shp")
ma_shp_dir = os.path.join(base_dir, "tl_2025_ma_zcta520")
ma_shp_path = os.path.join(ma_shp_dir, "tl_2025_ma_zcta520.shp")

# Create output directory
os.makedirs(ma_shp_dir, exist_ok=True)

# Load shapefiles
print("Loading ZCTA shapefile...")
zcta = gpd.read_file(zcta_shp)

print("Loading US States shapefile...")
states = gpd.read_file(states_shp)

# Filter Massachusetts
ma = states[states["STUSPS"] == "MA"]

# Ensure CRS match
if zcta.crs != ma.crs:
    ma = ma.to_crs(zcta.crs)

# Spatial join: keep ZCTAs that intersect Massachusetts
print("Performing spatial join to extract MA ZCTAs...")
ma_zcta = gpd.sjoin(zcta, ma, how="inner", predicate="intersects")

# Keep only original ZCTA columns
ma_zcta = ma_zcta[zcta.columns].copy()
ma_zcta.reset_index(drop=True, inplace=True)

# Save MA ZCTA shapefile
ma_zcta.to_file(ma_shp_path)
print(f"Massachusetts ZCTA shapefile saved to: {ma_shp_path}")

# Optional: delete original directories to save space (confirm MA shapefile first!)
shutil.rmtree(os.path.join(base_dir, "tl_2025_us_zcta520"))
shutil.rmtree(os.path.join(base_dir, "tl_2025_us_state"))
print("Deleted original US shapefile directories to save space.")

In [2]:
# --------------------------------------------------------------------------------------------------
# Cell: Download Massachusetts Tracts shapefile
#
# This cell downloads the full Massachusetts Tracts shapefile for 2025 from the
# US Census Bureau TIGER/Line repository, extracts it locally, and prepares it for further analysis.
# --------------------------------------------------------------------------------------------------


download_and_extract_tiger(
    input_url="TRACT/tl_2025_25_tract.zip",
    output_path="../data/external/tl_2025_ma_tract",
)

Downloading: https://www2.census.gov/geo/tiger/TIGER2025/TRACT/tl_2025_25_tract.zip
Download complete.
Extracting...
Extracted to: ../data/external/tl_2025_ma_tract
ZIP file removed.


| Field        | Description |
|--------------|-------------|
| ZCTA5CE20    | The 5-digit ZIP Code Tabulation Area code. |
| GEOID20      | Unique identifier for the ZCTA, usually identical to ZCTA5CE20. |
| GEOIDFQ20    | Fully-qualified GEOID, sometimes including state or other prefix information. |
| CLASSFP20    | Class code describing the type of ZCTA (typically “ZCTA5”). |
| MTFCC20      | MAF/TIGER Feature Class Code, categorizing the geographic feature. |
| FUNCSTAT20   | Functional status code (e.g., active or inactive ZCTA). |
| ALAND20      | Land area of the ZCTA in square meters. |
| AWATER20     | Water area of the ZCTA in square meters. |
| INTPTLAT20   | Latitude of the ZCTA’s internal point (centroid). |
| INTPTLON20   | Longitude of the ZCTA’s internal point (centroid). |
| geometry     | Polygon geometry defining the ZCTA boundaries. |


In [ ]:
# # Pull Massachusetts from the US States
# # --------------------------------------------------------------------------------------------------

# import geopandas as gpd
# extract_dir = "../data/external/tl_2025_us_state"
# shp_path = os.path.join(extract_dir, "tl_2025_us_state.shp")

# us_states = gpd.read_file(shp_path)

# print(len(us_states))
# # Filter for Massachusetts
# ma = us_states[us_states['STUSPS'] == 'MA']

# ma

In [3]:
place_shp = "../data/external/tl_2025_ma_place/tl_2025_25_place.shp"
places_gdf = gpd.read_file(place_shp)

# Filter Worcester
worcester = places_gdf[places_gdf["NAME"].str.upper() == "WORCESTER"]
print(worcester.geometry)
coords = list(worcester.geometry.iloc[0].exterior.coords)
print(coords)

18    POLYGON ((-71.88404 42.28125, -71.88387 42.281...
Name: geometry, dtype: geometry
[(-71.884043, 42.281253), (-71.883871, 42.281358), (-71.883161, 42.281792), (-71.880684, 42.283306), (-71.880377, 42.283498), (-71.878871, 42.284427), (-71.876791, 42.285685), (-71.875388, 42.286544), (-71.875238, 42.286636), (-71.872601, 42.288249), (-71.872513, 42.288299), (-71.872066, 42.288575), (-71.871082, 42.289177), (-71.870269, 42.289674), (-71.868232, 42.290919), (-71.866633, 42.291896), (-71.865221, 42.292762), (-71.863951, 42.293538), (-71.863627, 42.293732), (-71.863134, 42.294037), (-71.862589, 42.294364), (-71.862138, 42.294643), (-71.859062, 42.296523), (-71.856096, 42.298336), (-71.850608, 42.30169), (-71.845416, 42.304859), (-71.840081, 42.308122), (-71.839919, 42.308222), (-71.837764, 42.309539), (-71.837712, 42.30957), (-71.836972, 42.310021), (-71.834964, 42.31125), (-71.833861, 42.311923), (-71.831829, 42.313164), (-71.828863, 42.314981), (-71.828637, 42.315113), (-71.827616, 4

In [ ]:
lons, lats = zip(*coords, strict=True)

# Compute bounding box
min_lon, max_lon = min(lons), max(lons)
min_lat, max_lat = min(lats), max(lats)

bbox = (min_lon, min_lat, max_lon, max_lat)
print("Bounding box (min_lon, min_lat, max_lon, max_lat):", bbox)

Bounding box (min_lon, min_lat, max_lon, max_lat): (-71.884043, 42.210053, -71.731237, 42.341187)


In [3]:
# # Print all columns (fields) in the shapefile
# import os

# import geopandas as gpd

# extract_dir = "../data/external/tl_2025_us_zcta520"
# shp_path = os.path.join(extract_dir, "tl_2025_us_zcta520.shp")

# zcta = gpd.read_file(shp_path)

# # Print all column names
# print("Columns in the shapefile:")
# print(list(zcta.columns))

# # Optionally, inspect the first few rows
# print("\nSample rows:")
# print(zcta.head())

In [ ]:
# Print all columns (fields) in the shapefile
import os

import geopandas as gpd

extract_dir = "../data/external/tl_2025_ma_zcta520"
shp_path = os.path.join(extract_dir, "tl_2025_ma_zcta520.shp")

zcta = gpd.read_file(shp_path)

# Print all column names
print("Columns in the shapefile:")
print(list(zcta.columns))

# Optionally, inspect the first few rows
print("\nSample rows:")
print(zcta.head())

In [ ]:
import geopandas as gpd

# Load ZCTA shapefile
zcta_shp = "../data/external/tl_2025_ma_zcta520/tl_2025_ma_zcta520.shp"
zcta_gdf = gpd.read_file(zcta_shp)

# Ensure ZIPs are strings
zcta_gdf["ZCTA5CE20"] = zcta_gdf["ZCTA5CE20"].astype(str)

# Filter ZIPs starting with "016"
zips_016 = (
    zcta_gdf[zcta_gdf["ZCTA5CE20"].str.startswith("016")]["ZCTA5CE20"]
    .drop_duplicates()
    .sort_values()
    .tolist()
)
zips_016

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Assuming places_gdf is already loaded
# Filter Worcester
worcester = places_gdf[places_gdf["NAME"].str.upper() == "WORCESTER"]

# Plot all places in gray
places_gdf.plot(color="lightgray", edgecolor="black", figsize=(10, 10))

# Overlay Worcester in red
worcester.plot(color="red", edgecolor="black", alpha=0.7)

plt.title("Worcester Geometry Overlay")
plt.show()

NameError: name 'places_gdf' is not defined

In [4]:
import folium

# Project Worcester to a local projected CRS (meters)
# EPSG:26986 is Massachusetts Mainland / NAD83 (good for Worcester area)
worcester_proj = worcester.to_crs(epsg=26986)

# Compute centroid in projected CRS
centroid_proj = worcester_proj.geometry.centroid.iloc[0]

# Convert centroid back to lat/lon for folium
centroid_latlon = (
    gpd.GeoSeries([centroid_proj], crs=worcester_proj.crs)
    .to_crs(epsg=4326)
    .geometry.iloc[0]
)

# Create folium map
m = folium.Map(
    location=[centroid_latlon.y, centroid_latlon.x],
    zoom_start=12,
    tiles="OpenStreetMap",
)

# Add Worcester boundary
folium.GeoJson(
    worcester,
    name="Worcester",
    style_function=lambda feature: {
        "fillColor": "red",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.5,
    },
).add_to(m)

folium.LayerControl().add_to(m)
m